In [ ]:
import torch
import sys
sys.path.append('F:\\nmrprediction\\CSpre\\inmemory')
from model import regression
import math
import matplotlib.pyplot as plt
import torch
from bmrb_seq_dataset import refdb_get_seq, refdb_find_shift, refdb_get_shift_re, refdb_get_cs_seq
from pdb_bmrb_ali import align_bmrb_pdb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import esm
import numpy as np
from scipy import stats
import os

In [ ]:
def single_test_re(file_path, atom_type):
    bmrb_seq = refdb_get_seq(file_path)
    s, e = refdb_find_shift(file_path)
    cs_seq = refdb_get_cs_seq(file_path, s, e)
    matched = align_bmrb_pdb(bmrb_seq, cs_seq)
    shift, mask = refdb_get_shift_re(file_path, s, e, bmrb_seq, matched, atom_type)
    if '_' not in bmrb_seq:
        label= torch.tensor(shift)
        mask = torch.tensor(mask)
        label = torch.nn.functional.pad(label, (0, 512-label.shape[0]))
        model_path = "F:\\.cache\\torch\\hub\\checkpoints\\esm2_t33_650M_UR50D.pt"
        model, alphabet = esm.pretrained.load_model_and_alphabet(model_path)
        batch_converter = alphabet.get_batch_converter()
        model.eval()
        data = [("protein1", bmrb_seq)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33], return_contacts=False)
        token_representations = results["representations"][33]
        embedding = token_representations[:, 1:-1, :].squeeze()
        padding_mask = torch.zeros(512).bool()
        padding_mask[:embedding.shape[0]] = True
        embedding = torch.nn.functional.pad(embedding, (0, 0, 0, 512 - embedding.shape[0]))
        mask = torch.nn.functional.pad(mask, (0, 512 - mask.shape[0]), value=False)
        model = regression(1280, 512, 8, 0.1)
        # model = classification(1280, 512, 8, 0.1)
        padding_mask = padding_mask.unsqueeze(0)
        model_ckpt = {"CA":"reg_ca.pth", "CB":"reg_cb.pth", "C":"reg_c.pth", "N":"reg_n.pth", "HA":"reg_ha.pth", "H":"reg_h.pth"}
        model.load_state_dict(
            torch.load("F:\\nmrprediction\\CSpre\\inmemory\\best_model\\"+model_ckpt[atom_type], map_location=torch.device('cpu')))
        model.eval()
        out = model(embedding.unsqueeze(0), padding_mask)
        loss_func = torch.nn.MSELoss()
        loss = loss_func(out.squeeze(2).squeeze(0)[mask], label[mask])
        return math.sqrt(loss), out.squeeze(2).squeeze(0)[mask].detach(), label[mask].detach()
    return False, False, False